In [ ]:
%matplotlib notebook

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
import os
os.chdir('../')
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from particle_tracker_one_d.particle_tracker import ParticleTracker
from particle_tracker_one_d.shortest_path_finder import ShortestPathFinder

# Quick start

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from particle_tracker_one_d import ParticleTracker

# Import the frames and the time data
frames = np.load('examples/frames.npy')
time = np.load('examples/time.npy')

# Normalise the intensity
frames_normalised = ParticleTracker.normalise_intensity(frames)

# Create a particle tracker instance
pt = ParticleTracker(frames=frames_normalised, time=time)

# Set the properies of the particle tracker
pt.boxcar_width = 10
pt.integration_radius_of_intensity_peaks = 25
pt.particle_detection_threshold = 0.9
pt.maximum_number_of_frames_a_particle_can_disappear_and_still_be_linked_to_other_particles = 5
pt.maximum_distance_a_particle_can_travel_between_frames = 50


# Create a figure
plt.figure(figsize=(10,10))
ax=plt.axes()

# Plot the kymograph
pt.plot_all_frames(ax=ax, aspect='auto')

# Plot all the trajectories
for t in pt.trajectories:
    t.plot_trajectory(ax=ax, marker='o')

# Plot one trajectory

In [ ]:
length_trajectory = 0
index_of_longest_trajectory = 0
for index, t in enumerate(pt.trajectories):
    if t.particle_positions.shape[0] > length_trajectory:
        length_trajectory = t.particle_positions.shape[0]
        index_of_longest_trajectory = index

        
trajectory = pt.trajectories[index_of_longest_trajectory]



# Create a figure
plt.figure(figsize=(10,10))
ax=plt.axes()

# Plot the kymograph
pt.plot_all_frames(ax=ax, aspect='auto')


trajectory.plot_trajectory()

In [ ]:
fig = plt.figure(figsize=(5,15))
ax = plt.axes()
pt.plot_all_frames(ax=ax, aspect='auto')

start_point = (50, 75)
end_point = (159,265)

ax.plot(start_point[1], start_point[0],marker='o')
ax.plot(end_point[1], end_point[0],marker='o')
fig.tight_layout()

# Find shortest path

In [ ]:
import cProfile

In [ ]:
# Import the frames and the time data
frames = np.load('examples/frames.npy')
time = np.load('examples/time.npy')

# Normalise the intensity
frames_normalised = ParticleTracker.normalise_intensity(frames)

pt = ParticleTracker(frames_normalised,time)

# Create a shortest path finder instance
spf = ShortestPathFinder(frames=frames_normalised, time=time)

# Set the properies of the path finder
spf.boxcar_width = 20
spf.integration_radius_of_intensity_peaks = 10

# Set start point
spf.start_point = start_point

# Set end point
cProfile.run('spf.end_point = end_point')
#cProfile.run('spf.end_point = (22,140)')

In [ ]:
t = spf.trajectory
#t._particle_positions['frame_index'] = t._particle_positions['frame_index'] - t._particle_positions['frame_index'][0]

fig = plt.figure(figsize=(5,15))
ax = plt.axes()


pt.plot_all_frames(ax=ax, aspect='auto')
#ax.imshow(spf._intensity_of_interest,aspect='auto')

t.plot_trajectory(ax=ax, marker='o')
fig.tight_layout()

In [ ]:
spf._cost_matrix[-1]

In [ ]:
particles = [p.shape[0] for p in spf._particle_positions]
particles

In [ ]:
21*28*35*25*27*22

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(spf._intensity_of_interest[1])